In [281]:
np.asarray("A", dtype = 'double')

ValueError: could not convert string to float: 'A'

In [319]:
np.uint8('a')

ValueError: invalid literal for int() with base 10: 'a'

In [329]:
A = 'fooBar...'

In [328]:
c = np.array([np.str(ord(char)) for char in A])
print(c.dtype)
c

<U3


<ipython-input-328-af060ef61ac2>:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  c = np.array([np.str(ord(char)) for char in A])


array(['102', '111', '111', '66', '97', '114', '46', '46', '46'],
      dtype='<U3')

In [296]:
np.array(["GG"]).view(np.int32)[0]

71

In [1]:
import numpy as np

In [190]:
def rankWithTies(A, method, *args):
    #stores calls to built in methods
    DEFAULT_METHODS = {'min': lambda rMax, nMatch : rMax - (nMatch-1),
                       'max': lambda rMax, nMatch : rMax,
                       'random': lambda rMax, nMatch : np.random.permutation(nMatch) + (rMax - (nMatch - 1))
                      }
    
    #if no dimension specified, assume ranking along last dimension of A
    if not len(args):
        args += (-1,)
        
    dim = args[0]
        
    I = np.argsort(A, *args)
    #avoids sorting twice
    B = np.take_along_axis(A, I, dim)
    
    #initialization of inverse index array
    R = np.zeros_like(A)

    N = A.shape[dim]

    #reshapes the arange(N) array to be along the axis to insert in R 
    indexer = tuple([None if i!=dim else Ellipsis for i in range(A.ndim)])
    
    #np take_along_index is the correct np function to map (A,I) to B and (B,R) to A
    np.put_along_axis(R, I, np.arange(N, dtype = float)[indexer], axis=dim)
    
    if method == 'stable':
        return R, B, I
    
    elif method in DEFAULT_METHODS.keys():
        fun = DEFAULT_METHODS[method]
        
    elif type(method) == 'function':
        fun = method
        
    else: 
        raise ValueError(f"Invalid method type: {method}")
    
    #move axis to loop through to end for ease of iteration
    B_shift = np.moveaxis(B, dim, -1)
    I_shift = np.moveaxis(I, dim, -1)
    R_shift = np.moveaxis(R, dim, -1)

    for index in np.ndindex(B_shift.shape[:-1]):
        R_shift[index] = replaceDuplicateRanks(B_shift[index], I_shift[index], R_shift[index], fun)

    return R, B, I

In [127]:
def replaceDuplicateRanks(B, I, R, fun):
    
    #arrays with the indices the unique values start/stop at and their distances
    uniq, uniq_ind, uniq_count = np.unique(B, return_index = True, return_counts = True)
    
    #makes sure the to check for the last value being a duplicate
    uniq_ind = np.append(uniq_ind, len(B))
    
    for i in range(len(uniq_ind)-1):
        if uniq_count[i] != 1:
            #sets the rank to be a function of the index it last appears in in the sorted array
            
            #temp comment: uniq_ind[i+1]-1 s.t. input is the last index the val appears in
            #    but uniq_count[i]-1 s.t. the input is one Minus the # repeats (dist btwn max and min)
            #    does this need changing?
            R[I[uniq_ind[i]:uniq_ind[i+1]]] = fun(uniq_ind[i+1]-1, uniq_count[i])
        
    return R

In [320]:
A = np.random.randint(0,4,[10,10,10])
A

array([[[2, 2, 3, 1, 0, 1, 0, 0, 2, 3],
        [0, 3, 0, 0, 2, 0, 2, 2, 2, 0],
        [3, 1, 1, 3, 2, 1, 2, 1, 1, 1],
        [2, 3, 1, 1, 1, 2, 1, 0, 0, 0],
        [1, 3, 3, 1, 1, 3, 0, 3, 1, 1],
        [1, 0, 2, 3, 3, 1, 2, 1, 1, 3],
        [0, 0, 3, 2, 0, 3, 2, 1, 2, 1],
        [1, 2, 0, 0, 3, 0, 2, 3, 1, 1],
        [2, 2, 1, 1, 0, 0, 3, 3, 3, 2],
        [0, 0, 2, 0, 3, 2, 3, 0, 0, 3]],

       [[1, 1, 1, 3, 2, 2, 1, 1, 1, 1],
        [3, 3, 3, 3, 3, 2, 3, 0, 3, 0],
        [1, 0, 0, 0, 2, 1, 0, 0, 1, 3],
        [0, 3, 2, 1, 2, 3, 2, 0, 3, 2],
        [3, 3, 3, 0, 0, 2, 1, 3, 1, 1],
        [0, 2, 2, 0, 0, 3, 2, 3, 1, 2],
        [2, 0, 0, 0, 1, 0, 0, 1, 2, 1],
        [0, 1, 2, 1, 2, 0, 1, 3, 3, 3],
        [0, 2, 3, 3, 3, 1, 1, 0, 1, 1],
        [2, 3, 3, 2, 3, 0, 0, 3, 3, 0]],

       [[3, 3, 0, 1, 3, 0, 1, 0, 3, 3],
        [2, 1, 1, 1, 2, 1, 0, 2, 0, 1],
        [1, 1, 0, 0, 0, 3, 0, 0, 3, 1],
        [0, 0, 2, 2, 2, 0, 3, 3, 0, 2],
        [0, 0, 2, 3, 1, 3, 2, 3, 1, 

In [230]:
C = np.take_along_axis(A, I, dim)
np.all(C == B)

True

In [231]:
D = np.take_along_axis(B, R, dim)
np.all(D == A)

True

In [337]:
A[rankWithTies(A, 'min', 0)[0]]

IndexError: arrays used as indices must be of integer (or boolean) type

In [352]:
A

'fooBar'

In [365]:
np.ones([1, 5])

array([[1., 1., 1., 1., 1.]])

In [357]:
assert (B.shape == np.array([1,1,1,6])).all()

In [349]:
A = np/array'fooBar'
B = np.array(list(A)).reshape(1,1,1,len(A))
B

array([[[['f', 'o', 'o', 'B', 'a', 'r']]]], dtype='<U1')

In [371]:
A = np.array(list('fooBar...'))

for dimInd in range(0,3):
    #reshape A to be along different dimensions
    aSize = np.ones(max(2, dimInd+1), dtype = 'int64')
    print(len(A))
    aSize[dimInd] = len(A)
    A = A.reshape(aSize)

9
9
1


ValueError: cannot reshape array of size 9 into shape (1,1,1)

In [394]:
for i in np.arange(0, 6)[::-1]:
            print(i)

5
4
3
2
1
0


In [381]:
np.random.seed(42)
print(np.random.uniform(0,1,10))
np.random.seed(42)
print(np.random.uniform(0,1,10))

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258]
[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258]


In [274]:
AList = [np.array([np.pi, 1, 1, 2, 3, 3])]
methodList = ['min', 'max', 'random', 'stable', lambda rMax, nMatch : np.arange((rMax-(nMatch-1)), rMax+1)]
sortOptsList = [[0, 'mergesort'], []]

for A in AList:
    for method in methodList:
        for opts in sortOptsList:
            args = [A, method, *opts]
            print(args)
            
            R, B, I = rankWithTies(*args)
            print('Hello there')

[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'min', 0, 'mergesort']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'min']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'max', 0, 'mergesort']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'max']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'random', 0, 'mergesort']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'random']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'stable', 0, 'mergesort']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.        ,
       3.        ]), 'stable']
Hello there
[array([3.14159265, 1.        , 1.        , 2.        , 3.  

ValueError: Invalid method type: <function <lambda> at 0x7ff4a4031c10>

In [278]:
callable(args[1])

True

In [223]:
A[0,:,0]

array([1, 3, 1, 0, 1, 3, 1, 2, 3, 3])

In [224]:
R[0,:,0]

array([1, 6, 2, 0, 3, 7, 4, 5, 8, 9])

In [253]:
np.sort(A, 'descending')

TypeError: an integer is required (got type str)